In [1]:
import pandas as pd

In [2]:
with open('geo_data/cols.txt') as fp:
    headers = [line.split(':')[0].strip() for line in fp]
cols = ['name', 'country code', 'timezone', 'population']
df = pd.read_csv(filepath_or_buffer='geo_data/allCountries.txt',
                 sep='\t', names=headers, usecols=cols)


In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12368093 entries, 0 to 12368092
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   name          object
 1   country code  object
 2   population    int64 
 3   timezone      object
dtypes: int64(1), object(3)
memory usage: 2.5 GB


In [71]:
best_matches = df.sort_values(by='population', ascending=False).drop_duplicates(subset='name') \
.rename(columns={'country code':'country_code'}) \
.query('population > 15000 and not timezone.isna() and not country_code.isna()') \
.drop(columns='population') \
.query('name.str.fullmatch(r"[ -~]+")') \
.reset_index(drop=True)

In [77]:
best_matches.to_json(path_or_buf='geo_data/locations.json')

In [96]:
country = pd.read_csv('geo_data/country_info.txt', sep='\t', usecols=['ISO', 'Country', 'Continent'])

In [97]:
country

,ISO,Country,Continent
0,AD,Andorra,EU
1,AE,United Arab Emirates,AS
2,AF,Afghanistan,AS
3,AG,Antigua and Barbuda,NaN
4,AI,Anguilla,NaN
...,...,...,...
247,ZA,South Africa,AF
248,ZM,Zambia,AF
249,ZW,Zimbabwe,AF
250,CS,Serbia and Montenegro,EU
